실습 #1: langchain을 이용하여 사용자가 업로드한 pdf 파일을 근거로 답변하는 ChatGPT 어플리케이션을 작성하시오.

In [ ]:
!pip install langchain openai pypdf tiktoken faiss-cpu python-dotenv 
!pip install -U langchain-community
!pip install langchain-openai

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI # OpenAI 라이브러리의 최신 버전은 client 객체를 사용합니다.

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# OpenAI API 키 설정 및 클라이언트 초기화
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("OPENAI_API_KEY 환경 변수가 설정되지 않았습니다. .env 파일을 확인해주세요.")

client = OpenAI(api_key=api_key)

print("OpenAI API 클라이언트가 성공적으로 초기화되었습니다.")

OpenAI API 클라이언트가 성공적으로 초기화되었습니다.


In [9]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_pdf(file_path):
    """
    PDF 파일을 로드하고 텍스트를 분할합니다.
    """
    loader = PyPDFLoader(file_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    texts = text_splitter.split_documents(documents)
    print(f"총 {len(texts)}개의 텍스트 청크로 분할되었습니다.")
    return texts

pdf_file_path = "sample.pdf"
document_chunks = load_and_split_pdf(pdf_file_path)
print (document_chunks[0])

총 10개의 텍스트 청크로 분할되었습니다.
page_content='< 1 Page >본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.본 PDF는 샘플 페이지입니다. 아무나 쓰셔도 됩니다. PDF 테스트 등을 위해 만들었습니다.' metadata={'producer': 'Hancom PDF 1.3.0.443', 'creator': 'Hancom PDF 1.3.0.443', 'creationdate': '2020-06-21T09:52:01+09:00', 'author': '(주)한글과컴퓨터', 'moddate': '2020-06-21T09:52:01+09:00', 'pdfversion': '1.4', 'source': 'sample.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}


In [8]:
from langchain_openai import OpenAIEmbeddings # 최신 LangChain 버전에서는 이렇게 임포트합니다.
from langchain_community.vectorstores import FAISS # 최신 LangChain 버전에서는 이렇게 임포트합니다.

def create_vectorstore(texts, openai_api_key):
    """
    텍스트 청크를 임베딩하고 FAISS 벡터 저장소를 생성합니다.
    """
    # OpenAIEmbeddings 초기화 시 api_key를 직접 전달합니다.
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    vectorstore = FAISS.from_documents(texts, embeddings)
    print("FAISS 벡터 저장소가 성공적으로 생성되었습니다.")
    return vectorstore

if document_chunks:
    vector_store = create_vectorstore(document_chunks, api_key)
else:
    print("문서 청크가 없어 벡터 저장소를 생성할 수 없습니다. PDF 파일을 확인해주세요.")
    vector_store = None

FAISS 벡터 저장소가 성공적으로 생성되었습니다.


In [4]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI # 최신 LangChain 버전에서는 이렇게 임포트합니다.

def setup_qa_chain(vectorstore, openai_api_key):
    """
    질의응답 체인을 설정합니다.
    """
    # ChatOpenAI 초기화 시 api_key를 직접 전달합니다.
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key) # temperature=0으로 설정하여 일관된 답변 유도
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff", # 'stuff'는 모든 문서를 한 번에 LLM에 전달
        retriever=vectorstore.as_retriever(),
    )
    print("질의응답 체인이 성공적으로 설정되었습니다.")
    return qa_chain

if vector_store:
    qa_chain = setup_qa_chain(vector_store, api_key)
else:
    print("벡터 저장소가 없어 질의응답 체인을 설정할 수 없습니다.")
    qa_chain = None

질의응답 체인이 성공적으로 설정되었습니다.


In [10]:
if qa_chain:
    print("\n--- 챗봇과 대화 시작 ---")
    while True:
        user_question = input("질문을 입력하세요 (종료하려면 '종료' 입력): ")
        if user_question.lower() == '종료':
            print("챗봇을 종료합니다.")
            break
        
        if not user_question.strip():
            print("질문을 입력해주세요.")
            continue

        try:
            response = qa_chain.run(user_question)
            print(f"\n챗봇 답변: {response}\n")
        except Exception as e:
            print(f"오류 발생: {e}")
            print("API 키가 올바른지 확인하거나, PDF 내용이 질문에 충분한 정보를 제공하는지 확인해주세요.")
else:
    print("질의응답 체인이 초기화되지 않아 챗봇을 실행할 수 없습니다. 이전 단계를 확인해주세요.")


--- 챗봇과 대화 시작 ---

챗봇 답변: A PDF is a Portable Document Format file, commonly used for sharing documents in a consistent format that can be viewed on various devices.


챗봇 답변: I don't know.


챗봇 답변: Yes, the text provided indicates that the PDF is a sample page created for testing and more.


챗봇 답변: 네, 이 PDF는 연습용으로 만들어진 것입니다.


챗봇 답변: 네, 이 PDF 파일은 연습용으로 만들어진 것입니다.


챗봇 답변: 저는 종료할 수 없는 챗봇이에요. 계속해서 질문이나 도움이 필요하신 내용이 있으면 무엇이든 물어봐 주세요!

챗봇을 종료합니다.
